# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd

In [3]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data Quiz") \
    .getOrCreate()

In [59]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [9]:
# TODO: write your code to answer question 1
user_log.select("page").where(user_log.userId == "").dropDuplicates().sort("page").show()

+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [27]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.filter(user_log.userId == "").select(["page", "level", "ts", "auth", "itemInSession", "sessionId"]).where(user_log.auth != "Logged Out").sort("ts").show()

+-----+-----+-------------+-----+-------------+---------+
| page|level|           ts| auth|itemInSession|sessionId|
+-----+-----+-------------+-----+-------------+---------+
| Home| free|1513784350284|Guest|            0|     3011|
|About| free|1513784461284|Guest|            1|     3011|
+-----+-----+-------------+-----+-------------+---------+



In [28]:
user_log.filter(user_log.userId == "").select("sessionId").dropDuplicates().count()

186

In [31]:
user_log.filter(user_log.userId == "").select(["page", "level", "ts", "itemInSession", "sessionId"]).sort(desc("itemInSession")).show()

+-----+-----+-------------+-------------+---------+
| page|level|           ts|itemInSession|sessionId|
+-----+-----+-------------+-------------+---------+
| Home| paid|1513787722284|           66|     2293|
|Login| paid|1513763026284|           65|     1078|
| Home| paid|1513763025284|           64|     1078|
|Login| paid|1513729763284|           61|     5175|
| Home| paid|1513729762284|           60|     5175|
| Home| paid|1513830380284|           58|     6878|
| Home| paid|1513742804284|           57|     5478|
|Login| paid|1513811494284|           53|     6652|
| Home| paid|1513811493284|           52|     6652|
|Login| paid|1513842368284|           46|     5431|
| Home| paid|1513842367284|           45|     5431|
| Home| paid|1513750618284|           44|     1985|
|Login| paid|1513799227284|           44|     5731|
| Home| paid|1513799226284|           43|     5731|
|About| paid|1513738848284|           43|     3146|
| Home| paid|1513738847284|           42|     3146|
| Home| paid

In [47]:
user_log.filter(user_log.userId == "").select(["userId", "page", "level", "ts", "itemInSession", "sessionId"]).where(user_log.sessionId == "6050").show()

+------+-----+-----+-------------+-------------+---------+
|userId| page|level|           ts|itemInSession|sessionId|
+------+-----+-----+-------------+-------------+---------+
|      |Login| paid|1513782398284|            0|     6050|
|      | Home| paid|1513789890284|           37|     6050|
|      |Login| paid|1513789891284|           38|     6050|
+------+-----+-----+-------------+-------------+---------+



# Question 3

How many female users do we have in the data set?

In [48]:
# TODO: write your code to answer question 3
user_log.filter(user_log.gender == "F").select("userId").dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [52]:
# TODO: write your code to answer question 4
user_log.groupby(user_log.artist).count().orderBy(desc("count")).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [60]:
# TODO: write your code to answer question 5
ishome = udf(lambda x: 1 if x == "Home" else 0, IntegerType())

In [61]:
user_log = user_log.withColumn("ishome", ishome("page"))

In [62]:
user_log.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', ishome=0)

In [63]:
from pyspark.sql import Window

In [64]:
user_window = Window.partitionBy("userId").orderBy("ts").rangeBetween(Window.unboundedPreceding, 0)

In [65]:
user_log = user_log.withColumn("songcnt", Fsum("ishome").over(user_window))

In [80]:
user_log.select(["userId", "ts", "song", "page", "ishome", "songcnt"]).where(user_log.userId == "2756").sort("ts").show()

+------+-------------+--------------------+--------+------+-------+
|userId|           ts|                song|    page|ishome|songcnt|
+------+-------------+--------------------+--------+------+-------+
|  2756|1513780607284|Recipe For A Hear...|NextSong|     0|      0|
|  2756|1513782264284|                null|    Home|     1|      1|
|  2756|1513788285284|                null|    Home|     1|      2|
|  2756|1513788594284|            Has Been|NextSong|     0|      2|
|  2756|1513788730284|Shadowfax (2002 D...|NextSong|     0|      2|
|  2756|1513791207284|                null|    Help|     0|      2|
|  2756|1513831056284|          Knocked Up|NextSong|     0|      2|
|  2756|1513831486284|       Drunk On Love|NextSong|     0|      2|
+------+-------------+--------------------+--------+------+-------+



In [78]:
user_log.filter(user_log.page == 'NextSong').groupby("userId", "songcnt").agg({'songcnt':'count'}).show()

+------+-------+--------------+
|userId|songcnt|count(songcnt)|
+------+-------+--------------+
|  1436|      0|             2|
|  2088|      0|            13|
|  2162|      0|            15|
|  2162|      2|            19|
|  2294|      0|            11|
|  2294|      1|             4|
|  2294|      2|            16|
|  2294|      3|             3|
|  2294|      4|            17|
|  2294|      5|             4|
|  2904|      0|             1|
|   691|      0|             3|
|  1394|      0|            17|
|  1394|      1|             9|
|  2275|      1|             3|
|  2756|      0|             1|
|  2756|      2|             4|
|   451|      1|             1|
|   451|      2|             1|
|   800|      0|             2|
+------+-------+--------------+
only showing top 20 rows



In [79]:
user_log.filter(user_log.page == 'NextSong').groupby("userId", "songcnt").agg({'songcnt':'count'}).agg({'count(songcnt)':'avg'}).show()

+-------------------+
|avg(count(songcnt))|
+-------------------+
| 6.9558333333333335|
+-------------------+

